In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip --version

pip 25.0.1 from c:\Users\Krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip (python 3.12)

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 1. Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [ ]:

a = pd.read_csv('../raw_data_sets/aa_dataset-tickets-multi-lang-5-2-50-version.csv')
b = pd.read_csv('../raw_data_sets/dataset-tickets-multi-lang-4-20k.csv')

print(a.info())
print(b.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28587 entries, 0 to 28586
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   subject   24749 non-null  object
 1   body      28587 non-null  object
 2   answer    28580 non-null  object
 3   type      28587 non-null  object
 4   queue     28587 non-null  object
 5   priority  28587 non-null  object
 6   language  28587 non-null  object
 7   version   28587 non-null  int64 
 8   tag_1     28587 non-null  object
 9   tag_2     28574 non-null  object
 10  tag_3     28451 non-null  object
 11  tag_4     25529 non-null  object
 12  tag_5     14545 non-null  object
 13  tag_6     5874 non-null   object
 14  tag_7     2040 non-null   object
 15  tag_8     565 non-null    object
dtypes: int64(1), object(15)
memory usage: 3.5+ MB
None
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #  

In [7]:

# English only
a = a[a['language'] == 'en']
b = b[b['language'] == 'en']

# Drop nulls
a = a.dropna(subset=['priority', 'body'])
b = b.dropna(subset=['priority', 'body'])

# Keep only low, medium, high
valid_pr = ['low', 'medium', 'high']
a = a[a['priority'].isin(valid_pr)]
b = b[b['priority'].isin(valid_pr)]

# Create combined df with selected columns
s = pd.concat([a[['priority', 'body', 'subject']],
                      b[['priority', 'body', 'subject']]],
                     ignore_index=True)
s = s.sample(frac=1, random_state=42).reset_index(drop=True)


print(s.head())
print(s['priority'].value_counts())
print(s.shape)
# Save cleaned dataset
s.to_csv('../cleaned_data_sets/cleaned_email_priority_dataset.csv', index=False)


  priority                                               body  \
0   medium  customer support, our marketing agency is faci...   
1     high  Requesting comprehensive advice on optimizing ...   
2   medium  A user reported intermittent access issues wit...   
3     high  To the customer support team,\n\nIncident Deta...   
4     high  Facing disconnections after the software updat...   

                                             subject  
0                                                NaN  
1  Enhancing Investment Analytics with Microsoft ...  
2                                                NaN  
3  Urgent: Incident Report - Data Breach Affectin...  
4       Problem with Bose QuietComfort 35 II Headset  
priority
medium    11569
high      10917
low        5774
Name: count, dtype: int64
(28260, 3)


In [9]:
df=pd.read_csv('../cleaned_data_sets/cleaned_email_priority_dataset.csv')

In [10]:
df['priority'].value_counts()

priority
medium    11569
high      10917
low        5774
Name: count, dtype: int64

In [9]:
!pip list

Package                                  Version
---------------------------------------- -------------------
a2a-sdk                                  0.3.10
absl-py                                  1.4.0
accelerate                               1.9.0
aiobotocore                              2.25.1
aiofiles                                 22.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aioitertools                             0.12.0
aiosignal                                1.4.0
aiosqlite                                0.21.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.1
altair                                   5.5.0
annotated-types                          0.7.0
annoy                                    1.17.3
ansicolors                        

In [11]:
# Initialize tools (assuming these imports and initializations were done previously)
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define words to remove (priority terms)
PRIORITY_TERMS = r'\b(dear|support|team|ni|hello|hope|well|customer|data|please|message|team|nan|could|would|assistance|problem|issue|failure|system|update)\b'

def clean_text(text):
    # --- FIX for AttributeError: 'float' object has no attribute 'lower' ---
    # Convert the input (which might be NaN/float) to a string before processing.
    text = str(text)

    # 1. Convert to Lowercase
    text = text.lower()
    
    # 2. Remove Escape Sequences and Newline/Tab Characters (e.g., \n, \t, \a, \b)
    text = re.sub(r'[\r\n\t\a\b]+', ' ', text)
    
    # 3. Remove Priority/Severity Terms (New Step)
    # Uses the defined regex pattern to remove terms like 'critical', 'high', etc.
    # We use re.IGNORECASE in the pattern definition (r'\b(critical|...)\b') and then convert to lower, 
    # but since we convert to lower first, we only need the lowercase terms in the list.
    text = re.sub(PRIORITY_TERMS, '', text)
    
    # 4. Remove Punctuation and Symbols
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # 5. Remove leading/trailing/multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 6. Tokenization, Stop Word Removal, and Lemmatization
    tokens = text.split()
    
    # Remove Stop Words & Lemmatize (only if the resulting word is longer than 1 character)
    tokens = [
        lemmatizer.lemmatize(word) 
        for word in tokens 
        if word not in stop_words and len(word) > 1
    ]
    
    # Rejoin the tokens into a string
    return ' '.join(tokens)

In [ ]:
# Assuming your DataFrame is named 'df'
s['subject'] = s['subject'].apply(clean_text)
s['body'] = s['body'].apply(clean_text)

print("\n--- After Fixed Preprocessing Sample ---")
print(s[['subject', 'body']].head())
#preprocessed_data_set


--- After Fixed Preprocessing Sample ---
                                             subject  \
0                                                      
1  enhancing investment analytics microsoft azure...   
2                                                      
3  urgent incident report breach affecting patien...   
4                    bose quietcomfort 35 ii headset   

                                                body  
0  marketing agency facing challenge brand visibi...  
1  requesting comprehensive advice optimizing inv...  
2  user reported intermittent access issue projec...  
3  nincident detail incident description breach i...  
4  facing disconnection software already restarte...  


In [13]:
# Function to get the top N words
def get_top_words(cleaned_series, n=30):
    """Tokenizes a Series of cleaned text and returns the top N word counts."""
    all_text = ' '.join(cleaned_series)
    words = all_text.split()
    word_counts = Counter(words)
    return pd.DataFrame(word_counts.most_common(n), columns=['Word', 'Frequency'])

# Get top 30 words for Subject and Body
top_subject_words = get_top_words(s['subject'], n=30)
top_body_words = get_top_words(s['body'], n=30)

print("\n--- Top 10 Words in Subject ---")
print(top_subject_words.head(10))

print("\n--- Top 10 Words in Body ---")
print(top_body_words.head(10))


--- Top 10 Words in Subject ---
          Word  Frequency
0      medical       2713
1    analytics       2661
2      digital       2517
3     security       2476
4  integration       2422
5   investment       2244
6      concern       1861
7      request       1798
8      project       1782
9         tool       1730

--- Top 10 Words in Body ---
          Word  Frequency
0  information       9315
1      provide       9106
2     software       9009
3   appreciate       8821
4          due       7967
5     security       7273
6  integration       6883
7         tool       6810
8   investment       6657
9      greatly       6305


In [21]:
priority_map = {'low': 1, 'medium': 2, 'high': 3}
reverse_priority_map = {v: k for k, v in priority_map.items()}


X = s['body']
y = s['priority']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
#training dataset
train= pd.DataFrame({
    'body': X_train,
    'priority': y_train
})
#tesrting dataset
test= pd.DataFrame({
    'body': X_test,
    'priority': y_test
})
train.to_csv('../train_test_data_sets/train_email_priority_dataset.csv', index=False)
test.to_csv('../train_test_data_sets/test_email_priority_dataset.csv', index=False)

# # Convert text to numerical features using TF-IDF
# tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)